# Web Scraper for trip advisor

## Prerequisites

In [ ]:
pip install selenium

## Imports

In [4]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException

## Constants

In [5]:
URL_RESTAURANTS = "https://www.tripadvisor.com/Restaurants-g189473-Thessaloniki_Thessaloniki_Region_Central_Macedonia.html"
URL_ONE_CAFE = "https://www.tripadvisor.com/Restaurant_Review-g189473-d3807291-Reviews-To_Tsai_Thessaloniki-Thessaloniki_Thessaloniki_Region_Central_Macedonia.html"

PATH = "chromedriver.exe"

## Custom tools

#### Elements Class - Contains all the element identifiers

In [24]:
class element:
    '''
    This is a class that contains all the elements we will need to scrape data from TripAdvisor
    '''
    COOKIES_ACCEPT_BUTTON = (By.ID, "onetrust-accept-btn-handler")
    SHOW_MORE_ESTABLISHMENT_TYPES_BUTTON = (By.CLASS_NAME, "fdmYH")

    #Establishment Type elements:
    RESTAURANTS_ESTABLISHMENT_TYPE_CHECKBOX = (By.ID, 'checkbox_3')
    RESTAURANTS_ESTABLISHMENT_TYPE_BUTTON = (By.XPATH, f".//label[@for='{RESTAURANTS_ESTABLISHMENT_TYPE_CHECKBOX[1]}']")
    COFFEE_AND_TEA_ESTABLISHMENT_TYPE_CHECKBOX = (By.ID, 'checkbox_6')
    COFFEE_AND_TEA_ESTABLISHMENT_TYPE_BUTTON = (By.XPATH, f".//label[@for='{COFFEE_AND_TEA_ESTABLISHMENT_TYPE_CHECKBOX[1]}']")
    BARS_ESTABLISHMENT_TYPE_CHECKBOX = (By.ID, 'checkbox_241')
    BARS_ESTABLISHMENT_TYPE_BUTTON = (By.XPATH, f".//label[@for='{BARS_ESTABLISHMENT_TYPE_CHECKBOX[1]}']")


    LIST_ITEM = (By.XPATH,".//div[@data-test='&1_list_item']")
    LIST_ITEM_URL = (By.XPATH,".//a[@class='bHGqj Cj b']")
    REVIEW_TABLE = (By.ID, "taplc_location_reviews_list_resp_rr_resp_0")
    REVIEWS_COUNT = (By.CLASS_NAME, "reviews_header_count")
    REVIEW_CONTAINER = (By.XPATH,".//div[@class='review-container']")
    EXPAND_REVIEW_BUTTON = (By.XPATH,"//span[@class='taLnk ulBlueLinks']")

    REVIEW_TITLE = (By.XPATH,".//span[@class='noQuotes']")
    REVIEW_DATE = (By.XPATH,".//span[contains(@class, 'ratingDate')]")
    REVIEW_RATING = (By.XPATH,".//span[contains(@class, 'ui_bubble_rating bubble_')]")
    REVIEW_TEXT = (By.XPATH,".//p[@class='partial_entry']")
    NEXT_PAGE_IN_REVIEWS = (By.XPATH,'.//a[@class="nav next ui_button primary"]')

#### Establishments Class - Used to along with the select_establishments method of TripAdvisorScrapper

In [17]:
class establishment:
    def __init__(self,checkboxElement,buttonElement,clickWhenStateIs):
        self.checkboxElement = checkboxElement
        self.buttonElement = buttonElement
        self.clickWhenStateIs = clickWhenStateIs

#### TripAdvisor Scraper Class

In [27]:
class TripAdvisorScraper:
    def __init__(self, path, url):
        '''
        Constructor
        '''
        self.driver = webdriver.Chrome(PATH)
        self.url = url

    def getElementObjects(self, element, rootItem = None,placeHolderValues:list=None):
        '''
        This method is used to gather all matching elements
        '''
        if rootItem == None:
            rootItem = self.driver
        value = element[1]
        if placeHolderValues != None:
            for i in range(0,len(placeHolderValues)):
                value = value.replace("&" + str(i),placeHolderValues[i])
        return rootItem.find_elements(element[0],value)

    def getElementObject(self, element, rootItem = None,placeHolderValues:list=None):
        '''
        This method is used to gather an element defined in the elements class
        '''
        if rootItem == None:
            rootItem = self.driver
        value = element[1]
        if placeHolderValues != None:
            for i in range(0,len(placeHolderValues)):
                value = value.replace("&" + str(i),placeHolderValues[i])
        return rootItem.find_element(element[0],value)

    def waitForElement(self,element,rootItem = None, placeHolderValues:list=None):
        '''
        This method is used to gather an element defined in the elements class but it also waits until the element is presented
        '''
        if rootItem == None:
            rootItem = self.driver
        value = element[1]
        if placeHolderValues != None:
            for i in range(0,len(placeHolderValues)):
                value = value.replace("&" + str(i+1),placeHolderValues[i])
        try:
            return WebDriverWait(rootItem, 20).until(EC.presence_of_element_located((element[0],value)))
        except:
            return None

    def acceptCookies(self):
        try:
            cookies = self.waitForElement(element.COOKIES_ACCEPT_BUTTON)
            cookies.click()
        except:
            print("No cookies! Impolite! Meh...")

    def open_browser(self):
        '''
        This method is used to navigate to the URL that was instructed to the scraper durring initialization
        '''
        self.driver.get(self.url) #Load URL
        self.acceptCookies()

    def select_establishments(self,establishments:list):
        '''
        This method is used to select establishment types on TripAdvisor
        Parameters:
        establistmentElements :
            Gets a list of objects with three keys, checkbox, initialState and button :
                'checkbox'      : The checkbox element to check (Type: element object from element class)
                'button'        : The button element to click in order to change the state (Type: element object from element class)
                'initialState'  : The initial state of the checkbox (Type: boolean, True/False)
        '''

        #Expand list of establishments:
        try:
            #there are other elements with the same class name, but this is the first one
            more_button = self.waitForElement(element.SHOW_MORE_ESTABLISHMENT_TYPES_BUTTON)
            more_button.click()
        except:
            pass

        #Select establishment types:
        for e in establishments:
            if type(e) != establishment: continue
            try:
                establishment_checkbox = self.getElementObject(e.checkboxElement).is_selected()
                if establishment_checkbox == e.clickWhenStateIs:
                    self.getElementObject(e.buttonElement).click()
            except:
                pass

    def loop_at_rest(self):
        num_page = 2
        for i in range(0, num_page):
            num_items = 1
            for j in range(1, num_items+1):
                time.sleep(10)
                item = self.waitForElement(element.LIST_ITEM, placeHolderValues=[str(j)] )
                url = self.getElementObject(element.LIST_ITEM_URL, rootItem=item).get_attribute("href")
                self.driver.get(url)
                self.get_reviews()

    def get_reviews(self):
        self.acceptCookies()
        #num_page = 10
        review_table = self.waitForElement(element.REVIEW_TABLE)
        try:
            reviews_no = self.getElementObject(element.REVIEWS_COUNT).text
            reviews_no = int(reviews_no.replace("(", "").replace(")", ""))
            num_page = int(reviews_no / 10)
            print("page = ",num_page)
            k=0
            for i in range(0, num_page):
                # expand the review
                time.sleep(4) #<= We need to think about this waiting time {!}
                container = self.getElementObjects(element.REVIEW_CONTAINER)
                print("container ", len(container))
                for j in range(len(container)):
                    try:
                        try:
                            time.sleep(2) #<= We need to think about this waiting time {!}
                            more = self.getElementObject(element.EXPAND_REVIEW_BUTTON,rootItem=container[j])
                            if more != None:
                                if more.text == "More":
                                    more.click()
                        except StaleElementReferenceException:
                            print("Comment ",j, " could be longer, but I will let that pass..." )

                        title = self.getElementObject(element.REVIEW_TITLE,rootItem=container[j]).text
                        date = self.getElementObject(element.REVIEW_DATE,rootItem=container[j]).get_attribute("title")
                        rating = self.getElementObject(element.REVIEW_RATING,rootItem=container[j]).get_attribute("class").split("_")[3]
                        review = self.getElementObject(element.REVIEW_TEXT,rootItem=container[j]).text.replace("\n", " ")
                        print(title)
                        print(date)
                        print(rating)
                        print(review)
                        print("-----------")
                    except NoSuchElementException:
                        print("Didn't found item ",j, ", but it should be there.")
                k=i
                # change the page
                self.getElementObject(element.NEXT_PAGE_IN_REVIEWS).click()
        except NoSuchElementException:
            print("Oops, unlucky search!")
        except TimeoutException:
            print("You run out of time!")

        for i in (0,k-1):
            self.driver.execute_script("window.history.go(-1)")

    def quit_browser(self):
        self.driver.quit()


In [ ]:
TripAdvisorScraper = TripAdvisorScraper(PATH,URL_RESTAURANTS)
TripAdvisorScraper.open_browser()
TripAdvisorScraper.select_establishments(
    establishments=[
        establishment(element.RESTAURANTS_ESTABLISHMENT_TYPE_CHECKBOX,element.RESTAURANTS_ESTABLISHMENT_TYPE_BUTTON,True),
        establishment(element.COFFEE_AND_TEA_ESTABLISHMENT_TYPE_CHECKBOX,element.COFFEE_AND_TEA_ESTABLISHMENT_TYPE_BUTTON,False),
        establishment(element.BARS_ESTABLISHMENT_TYPE_CHECKBOX,element.BARS_ESTABLISHMENT_TYPE_BUTTON,False)
    ]
)
TripAdvisorScraper.loop_at_rest()
# TripAdvisorScraper.get_reviews()
TripAdvisorScraper.quit_browser()